In [1]:
DATAPATH = "../resources/"

import numpy as np
import json
from tqdm import tqdm 

# Enviroment Setup

In [2]:
# Distance Funcs

def cosine_dist(x, y):
    return np.dot(x, y) / (np.sqrt(np.dot(x, x)) * np.sqrt(np.dot(y, y)))

def euc_dist(x,y):
    return np.sqrt(np.sum((x - y) ** 2))

In [3]:
# Load embeddings 
embeddings_index = dict()

with open(f'{DATAPATH}glove.6B.300d.txt',encoding="utf8") as fp:
    for line in tqdm(fp,total=400000):
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

word_vec = embeddings_index

 53%|█████▎    | 210239/400000 [00:23<00:22, 8482.59it/s] 

KeyboardInterrupt: 

In [ ]:
len(word_vec)

In [ ]:
# Load game words
with open(f"{DATAPATH}lexicon.txt",'r') as fp:
    lexicon = json.load(fp)

# Game Functions

In [3]:
# utils

def strikeout(word):
    ''' Creates a strikethrough font in terminal '''
    newWord = []
    for char in word:
        newWord.append("\u0336"+char)
    return "".join(newWord)

# Distance Funcs
def cosine_dist(x, y):
    return np.dot(x, y) / (np.sqrt(np.dot(x, x)) * np.sqrt(np.dot(y, y)))

def euc_dist(x,y):
    return np.sqrt(np.sum((x - y) ** 2))

In [4]:
# exceptions

class InvalidBoardException(Exception):
    def __init__(self, message):
        super().__init__()
        self.message = "InvalidBoardException: " + message
    def __str__(self):
        return self.message
    
class InvalidGuessException(Exception):
    def __init__(self, message):
        super().__init__()
        self.message = "InvalidGuessException: " + message
    def __str__(self):
        return self.message

In [13]:
TGREEN =  '\033[32m'
TRED =  '\033[31m'
RESETC = '\033[m'

class Board(object):
    
    def __init__(self, game_words,targets, bombs):
        self.setGameWords(game_words)
        self.setTargets(targets)    
        self.setBombs(bombs)
        self.guessedWords = []
        
    def setGameWords(self, words):
        ''' setter for full list of game words'''
        set_size = len(words)
        if int((set_size**0.5 + 0.5)**2) != set_size:# round about way to verify square gridsize
            raise InvalidBoardException("Game words can not be fit into square grid") 
        if len(set(words)) != len(words):# make sure no duplicate words
            raise InvalidBoardException("Duplicate game words on board")
        self.words = words
            
    def setTargets(self, targets):
        ''' setter for target words '''
        if len(set(bombs)) != len(bombs):# make sure no duplicate words
            raise InvalidBoardException("Duplicate words in bomb set")
        self.targets = targets
    
    def setBombs(self, bombs):
        ''' setter for bomb words '''
        if len(set(bombs)) != len(bombs):# make sure no duplicate words
            raise InvalidBoardException("Duplicate words in bomb set")
        if any(word in bombs for word in self.targets):
            raise InvalidBoardException(f"Overlapping target and bomb sets\n{self.targets}\n{self.bombs}") 
        self.bombs = bombs
        
    def addGuess(self, guess):
        if guess not in self.getGameWords():
            raise InvalidGuessException("Guessed word is not on the board")
        if guess in self.getGuessWords():
            raise InvalidGuessException("Guessed word has already been guessed")
        self.guessedWords.append(guess)
    
    def getTargets(self, remaining=False):
        ''' getter for target words'''
        if remaining:# only return words that have not been cleared already
            idx = [word not in self.getGuessedWords for word in self.targets]
            return self.targets[idx]
        else:
            return self.targets
    
    def getBombs(self):
        ''' getter for bomb words'''        
        return self.bombs
    
    def getGameWords(self, active=False):
        ''' getter for current words on board'''
        if active:
            return [word for word in self.words if word not in self.getGuessedWords()]
        else:
            return self.words
    
    def getGuessedWords(self):
        ''' getter for all guessed words'''
        return self.guessedWords
    
    def __str__(self):
        ''' converts current gameboard to string format '''
        out = ''
        rowSize = int(len(self.words)**(0.5))
        for rowIdx in range(0, rowSize):
            for word in self.getGameWords[rowIdx*rowSize:rowIdx*rowSize+rowSize]:
                
                #create strikeout font if word has been guessed 
                if word in self.getGuessedWords():
                    writeWord = strikeout(word)
                else:
                    writeWord = word
                
                # colourize word based on word set
                if word in self.getTargets():
                    c = TGREEN
                elif word in self.bombs:
                    c = TRED
                else:
                    c = RESETC
                    
                out = "".join([out,c,'{0:<15}'.format(writeWord)])
            out = "".join([out,'\n'])
        return out

In [14]:
class Game(object):
    
    def __init__(self,board_dim = 5, target_count = 8, bomb_count = 3):
        self.gameword_lexicon = self.loadGameWords()
        setGameStructure(board_dim, target_count, bomb_count)
        self.resetBoard()
    
    def setGameStructure(board_dim, target_count, bomb_count):
        ''' define the parameters of the game '''
        self.board_dim = board_dim
        self.target_count = target_count
        self.bomb_count = bomb_count
    
    def resetBoard(self):
        ''' reset gameboard '''
        gamewords = random.choices(lexicon, k = self.board_dim**2)
        player_target, player_bomb = self.getTargetBombWords()
        comp_target, comp_bomb = self.getTargetBombWords()
        
        self.player_board = Board(gamewords, player_target, player_bomb)
        self.comp_board = Board(gamewords, comp_target, comp_bomb)
    
    def getTargetBombWords(self):
        ''' Randomly select target and bomb words from currect game_word sellection'''
        target_words = random.choices(self.gamewords, k=self.target_count)
        bomb_words = random.sample(set(self.gamewords).difference(set(self.target_words)), self.bomb_count)
        return target_words, bomb_words
        
    def loadGameWords(self):
        ''' loads list of possible game words '''
        DATAPATH = "resources/"
        with open(f"{DATAPATH}lexicon.txt",'r') as fp:
            gameword_lexicon = json.load(fp)
        return gameword_lexicon

    def getStrBoard(player):
        if player == 0:
            return str(self.getPlayerBoard())
        elif player == 1:
            return str(self.getComBoard())
        else:
            raise InvalidBoardException(f"No Board Exists for player {player}")
    # Getters
    def getPlayerBoard():
        return self.player_board
    def getCompBoard():
        return self.comp_board

In [15]:
class ComputerPlayer(object):
    
    def __init__(self, ):
        self.loadEmbeddings()
        
    def loadEmbeddings(self):
        ''' Prepares embeddings '''
        embeddings_index = dict()
        with open(f'{DATAPATH}glove.6B.300d.txt',encoding="utf8") as fp:
            for line in tqdm(fp,total=400000,position=0, leave=True):
                values = line.split()
                word = values[0]
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs
        self.word_vecs = embeddings_index
    
    def evaluateClue(current_board, clue_word, target_count):
        ''' Processes a clue provided from other player'''
        e_dist = []
        c_dist = []

        for word in current_board.getGameWords(active=True):
            c_dist.append((cosine_dist(self.word_vecs[word], self.word_vecs[clue_word]), word))
            e_dist.append((euc_dist(self.word_vecs[word], self.word_vecs[clue_word]), word))

        e_dist =sorted(e_dist, key=lambda x: x[0])
        c_dist =sorted(c_dist, key=lambda x: x[0], reverse=True)
        
        topWords = {}
        value = 1.0
        for eword, cword in zip(e_dist[:target_count+2], c_dist[:target_count+2]):
            topWords[eword] += value
            topWords[cword] += value*0.75
            value = max([0.2, value-0.2])
        selected_guesses = sorted(topWords.items(), key=lambda x: x[1])[target_count]
        
        return [guess[0] for guess in selected_guesses]
        
    def produceClue(current_bouard):
        ''' Creates a clue based on current game board '''
        pass

In [16]:
class GameMaster(object):
    
    def __init__(self, *args):
        print("Preparing Game...")
        self.ai = ComputerPlayer()
        self.setNewGame()

    def setNewGame(self):
        ''' sets board '''
        self.current_game = Game()
    
    def submitClue(clue_word, target_count):
        ''' sends clue to ai to predict associated target_words '''

        
    def getClue():
        ''' pulls clue from ai for current target words '''
        pass
    
    def drawBoard(player=0):
        return getStrBoard(player)

In [17]:
test = GameMaster()

  0%|          | 602/400000 [00:00<01:06, 6012.45it/s]

Preparing Game...


100%|██████████| 400000/400000 [00:59<00:00, 6773.56it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'resources/lexicon.txt'

# Game Setup

In [48]:
import random

BOARD_DIM = 5
TARGET_WORD_COUNT = 8
BOMB_WORD_COUNT = 3

TGREEN =  '\033[32m'
TRED =  '\033[31m'
RESETC = '\033[m'

In [49]:
session_words = random.choices(lexicon, k=BOARD_DIM**2)
target_words = random.choices(session_words, k=TARGET_WORD_COUNT)
bomb_words = random.sample(set(session_words).difference(set(target_words)), BOMB_WORD_COUNT)

In [50]:
for rowIdx in range(int(len(session_words)/BOARD_DIM)):
    for word in session_words[rowIdx*BOARD_DIM:rowIdx*BOARD_DIM + BOARD_DIM]:
        if word in target_words:
            c = TGREEN
        elif word in bomb_words:
            c = TRED
        else:
            c = RESETC
        print(c+'{0:<10}'.format(word), end='')
    print()

crust     spare     portfolio random    jelly     
scoundrel machine   lightsaberphilosopherelf       
sheep     regret    spy       coaster   mop       
cardboard asleep    gasoline  windmill  riddle    
forehead  pajamas   slump     dinosaur  fortnight 


In [18]:
query = 'liquid'

e_dist = []
c_dist = []

for w in session_words:
    c_dist.append((cosine_dist(word_vec[w], word_vec[query]), w))
    e_dist.append((euc_dist(word_vec[w], word_vec[query]), w))

e_dist =sorted(e_dist, key=lambda x: x[0])
c_dist =sorted(c_dist, key=lambda x: x[0], reverse=True)


print(query)

print("          euc dict    |      cosine dist")
print("********************************************")
for pair in zip(e_dist, c_dist):
    print(pair)

liquid
          euc dict    |      cosine dist
********************************************
((7.432392, 'water'), (0.46924633, 'water'))
((7.815875, 'drink'), (0.36121622, 'drink'))
((7.9930882, 'bag'), (0.30777472, 'bag'))
((8.357532, 'brand'), (0.24772772, 'beer'))
((8.363356, 'zero'), (0.22306392, 'chicken'))
((8.674251, 'portfolio'), (0.21704932, 'brand'))
((8.756122, 'beer'), (0.21394573, 'portfolio'))
((8.903857, 'chicken'), (0.21145968, 'zero'))
((9.026575, 'flotsam'), (0.11179688, 'marble'))
((9.214224, 'plow'), (0.09283333, 'flower'))
((9.328238, 'scar'), (0.09155338, 'dunk'))
((9.336437, 'pilot'), (0.08240295, 'pilot'))
((9.442704, 'worm'), (0.077028275, 'worm'))
((9.500146, 'flower'), (0.067718804, 'scar'))
((9.513062, 'marble'), (0.047251157, 'flotsam'))
((9.542031, 'windmill'), (0.044647463, 'old'))
((9.610797, 'crow'), (0.038094327, 'plow'))
((9.638541, 'dunk'), (0.013391069, 'duck'))
((9.668399, 'tutor'), (0.0011603528, 'windmill'))
((9.703111, 'old'), (-0.0031345936, '